## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


릿지가 기본이 되지만 쓰이는 특성이 몇 개뿐이라고 의심되면 라쏘나 엘라스티넷이 낫다. 이 모델들은 불필요한 특성의 가중치를 0으로 만들어주기 때문. 특성 수 가 훈련 샘플 수보다 많거나 특성 몇 개가 강하게 연관되어 있을 때는 보통 라쏘가 문제를 일으키므로 라쏘보다는 엘라스티넷을 선호

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

과적합이 일어난 것으로 추정 -> 조기종료를 통해 해결할 수 있음 

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

훈련 오차와 검증 오차가 거의 비슷하고 둘 다 높은 것은 과소적합 때문 

따라서 편향이 높은 것이기 때문에 모델의 복잡도를 높이기 위해 규제 하이퍼파라미터를 줄여야함

### **4. 다음과 같이 사용해야 하는 이유는?**
___

- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀

과대 적합을 감소시키기 위해. 모델의 가중치를 제한하기 가장 일반적인 방법이기 때문

- 릿지 회귀 대신 라쏘 회귀

쓰이는 특성이 몇 개뿐이라고 의심되는 경우

- 라쏘 회귀 대신 엘라스틱넷

특성 수가 훈련 샘플 수보다 많거나 특성 몇개가 강하게 연관되어 있을 때는 보통 라쏘가 문제를 일으키므로 라쏘보다는 엘라스티넷을 선호

-> 라쏘는 특성 수가 샘플 수보다 많으면 최대 n개의 특성을 선택. 또한 여러 특성이 강하게 연관되어 있으면 이들 중 임의의 특성 하나를 선택하기 때문

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [3]:
from sklearn import datasets

iris = datasets.load_iris()

X = iris["data"]
y = iris["target"]

In [25]:
import numpy as np

X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [26]:
np.random.seed(2024)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2)

> 이 알고리즘 대신에 직접 스플릿하기

In [27]:
test_ratio = 0.2
validation_ratio = 0.2

total_size = len(X_with_bias)
test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [18]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [19]:
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)

In [20]:
best_accuracy = 0
best_epoch = None
best_model = None
tolerance = 10  # 허용 가능한 연속적인 성능 하락 횟수
count = 0  # 연속적인 성능 하락 횟수 초기화

for epoch in range(1000):
    softmax_reg.fit(X_train, y_train)
    y_val_predict = softmax_reg.predict(X_val)
    accuracy = accuracy_score(y_val, y_val_predict)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch
        best_model = clone(softmax_reg)
        count = 0  # 새로운 최고 성능이 발생했으므로 연속 성능 하락 횟수 초기화
    else:
        count += 1
        if count == tolerance:
            print("Early stopping at epoch", epoch)
            break  # 조기 종료


Early stopping at epoch 10


In [21]:
best_accuracy

1.0

In [22]:
best_accuracy = 0
best_epoch = None
best_model = None

for epoch in range(1000):
    softmax_reg.fit(X_train, y_train)
    y_val_predict = softmax_reg.predict(X_val)
    accuracy = accuracy_score(y_val, y_val_predict)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch
        best_model = clone(softmax_reg)

In [23]:
best_accuracy

1.0

> 소프트 맥스 함수를 직접 정의해야함

타깃은 클래스 인덱스(0, 1 그리고 2)이지만 소프트맥스 회귀 모델을 훈련시키기 위해 필요한 것은 타깃 클래스의 확률입니다. 각 샘플에서 확률이 1인 타깃 클래스를 제외한 다른 클래스의 확률은 0입니다(다른 말로하면 주어진 샘플에 대한 클래스 확률이 원-핫 벡터입니다). 클래스 인덱스를 원-핫 벡터로 바꾸는 간단한 함수를 작성하겠습니다

In [28]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [29]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [30]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [31]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))  

n_iterations만큼의 반복 실행


각 반복에서는 다음과 같은 작업을 수행
- 입력 데이터에 대한 가중치와의 내적을 통해 예측 점수(logits)를 계산합니다.
- 소프트맥스 함수를 사용하여 예측 클래스의 확률 분포(Y_proba)를 계산합니다.
- 손실 함수를 사용하여 그래디언트를 계산

그래디언트는 손실 함수를 각 파라미터에 대해 미분한 값으로, 파라미터를 업데이트하는 데 사용
계산된 그래디언트를 사용하여 파라미터 행렬 Theta를 업데이트합니다. 이때 학습률 eta를 사용하여 그래디언트의 크기를 조절합니다.

In [44]:
eta = 0.01
n_iterations = 10000
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

In [45]:
Theta

array([[ 1.05741137,  0.18649144, -2.34034035],
       [ 0.69889081,  0.21518329, -2.11131934],
       [ 1.89902633,  0.13194926, -1.20600382],
       [-3.4079844 ,  0.17898683,  3.39152256],
       [-0.32890627, -1.46337249,  1.56010421]])

In [46]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

In [47]:
eta = 0.01
n_iterations = 10000
m = len(X_train)
epsilon = 1e-7
best_loss = np.infty
tolerance = 1e-5  # 허용 오차 설정

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    
    loss = -1/m * np.sum(Y_train_one_hot * np.log(Y_proba + epsilon))
    
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients
    
    if loss < best_loss - tolerance:  # 최근 손실값과 이전 손실값의 차이가 허용 오차 이내로 감소하지 않으면
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "조기 종료!")
        break



6192 0.14735336097167556
6193 0.14734336354611458 조기 종료!


In [48]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0